# Price Optimization

This example is adapted from the example in Gurobi's modeling examples [How Much
Is Too Much? Avocado Pricing and Supply Using Mathematical
Optimization](https://github.com/Gurobi/modeling-examples/tree/master/price_optimization).

We develop the same example as in the documentation but we try and compare different
regression models to estimate demand.

Note that we remove the year as a feature because it doesn't play well with SKlearn decision trees.

This is mainly for testing that it works, the detailed results were not looked at and some estimators may give bogus solutions.

In [ ]:
import pandas as pd
import numpy as np

import gurobipy as gp
from gurobipy import GRB

from gurobi_ml import add_predictor_constr
import gurobipy_pandas as gppd

## Load the Packages and the Datasets

In [ ]:
# Get the data

data_url = "https://raw.githubusercontent.com/Gurobi/modeling-examples/master/price_optimization/"
avocado = pd.read_csv(
    data_url + "HABdata_2019_2022.csv"
)  # dataset downloaded directly from HAB
avocado_old = pd.read_csv(
    data_url + "kaggledata_till2018.csv"
)  # dataset downloaded from Kaggle
avocado = pd.concat([avocado, avocado_old])

# Add the index for each year from 2015 through 2022
avocado["date"] = pd.to_datetime(avocado["date"])
avocado["year"] = pd.DatetimeIndex(avocado["date"]).year
avocado["year_index"] = avocado["year"] - 2015
avocado = avocado.sort_values(by="date")

# Define the peak season
avocado["month"] = pd.DatetimeIndex(avocado["date"]).month
peak_months = range(2, 8)  # <--------- Set the months for the "peak season"


def peak_season(row):
    return 1 if int(row["month"]) in peak_months else 0


avocado["peak"] = avocado.apply(lambda row: peak_season(row), axis=1)

# Scale the number of avocados to millions
avocado["units_sold"] = avocado["units_sold"] / 1000000

# Select only conventional avocados
avocado = avocado[avocado["type"] == "Conventional"]

avocado = avocado[
    ["date", "units_sold", "price", "region", "year", "month", "year_index", "peak"]
].reset_index(drop=True)

avocado

## Train regressions

We prepare the data using `OneHotEncoder` and `make_column_transformer`. We want
to transform the region feature using the encoder while we apply scaling to the other features.

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_transformer

feat_transform = make_column_transformer(
    (OneHotEncoder(drop="first"), ["region"]),
    (StandardScaler(), ["price"]),
    ("passthrough", ["peak"]),
    verbose_feature_names_out=False,
    remainder='drop'
)


regions = [
    "Great_Lakes",
    "Midsouth",
    "Northeast",
    "Northern_New_England",
    "SouthCentral",
    "Southeast",
    "West",
    "Plains",
]
df = avocado[avocado.region.isin(regions)]

X = df[["region", "price", "peak"]]
y = df["units_sold"]

To validate the regression model, we will randomly split the dataset into $80\%$
training and $20\%$ testing data and learn the weights using `Scikit-learn`.

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data for training and testing
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.8, random_state=1
)

Create dictionary with various regression models that we want to use

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.base import clone
from xgboost import XGBRegressor
from time import time
args={"random_state":1}
regressions = {"Linear Regression": {"regressor":LinearRegression() },
               "MLP Regression": {"regressor": MLPRegressor([8]*2, max_iter=1000, **args)},
               "Decision Tree": {"regressor": DecisionTreeRegressor(max_leaf_nodes=50, **args)},
               "Random Forest": {"regressor": RandomForestRegressor(n_estimators=10, max_leaf_nodes=100, **args)},
               "Gradient Boosting":
               {"regressor" : GradientBoostingRegressor(n_estimators=20, **args)},
              "XGB Regressor": {"regressor": XGBRegressor(n_estimators=20, **args)}}

# Add polynomial features for linear regression and MLP
regressions_poly = {}
for regression in ["Linear Regression", "MLP Regression"]:
    data = {"regressor": (PolynomialFeatures(),clone(regressions[regression]["regressor"]))}
    regressions_poly[f"{regression} polynomial feats"] = data
# Merge dictionary of polynomial features
regressions |= regressions_poly

Train the regressions

In [ ]:
for regression, data in regressions.items():
    regressor = data["regressor"]
    if isinstance(regressor, tuple):
        lin_reg = make_pipeline(feat_transform,
                                *regressor)
    else:
        lin_reg = make_pipeline(feat_transform,
                                regressor)
    train_start = time()
    lin_reg.fit(X_train, y_train)
    data[("Learning","time")] = time() - train_start
    data["pipeline"] = lin_reg

    # Get R^2 from test data
    y_pred = lin_reg.predict(X_test)
    r2_test = r2_score(y_test, y_pred)
    y_pred = lin_reg.predict(X_train)
    r2_train = r2_score(y_train, y_pred)
    data[("Learning", "R2 test")] = r2_test
    data[("Learning", "R2 train")] = r2_train
    print(f"{regression:<18} R^2 value in the test set is {r2_test:.3f} training {r2_train:.3f}")

## Prepare data of optimization model

In [ ]:
# Sets and parameters
B = 30  # total amount ot avocado supply

peak_or_not = 1  # 1 if it is the peak season; 1 if isn't

c_waste = 0.1  # the cost ($) of wasting an avocado
# the cost of transporting an avocado
c_transport = pd.Series(
    {
        "Great_Lakes": 0.3,
        "Midsouth": 0.1,
        "Northeast": 0.4,
        "Northern_New_England": 0.5,
        "SouthCentral": 0.3,
        "Southeast": 0.2,
        "West": 0.2,
        "Plains": 0.2,
    }, name='transport_cost'
)

c_transport = c_transport.loc[regions]
# the cost of transporting an avocado

# Get the lower and upper bounds from the dataset for the price and the number of products to be stocked
a_min = 0  # minimum avocado price in each region
a_max = 2  # maximum avocado price in each region

data = pd.concat([c_transport,
                  df.groupby("region")["units_sold"].min().rename('min_delivery'),
                  df.groupby("region")["units_sold"].max().rename('max_delivery')], axis=1)

In [ ]:
data

In [ ]:
m = gp.Model("Avocado_Price_Allocation")

p = gppd.add_vars(m, data, name="price", lb=a_min, ub=a_max)
d = gppd.add_vars(m, data, name="demand") # Add variables for the regression
w = m.addVar(name="w") # excess wasteage
m.update()

m.setObjective((p * d).sum() - c_waste * w - (c_transport * d).sum())
m.ModelSense = GRB.MAXIMIZE

m.addConstr(d.sum() + w == B)
m.update()

In [ ]:
feats = pd.DataFrame(
    data={
        "peak": peak_or_not,
        "region": regions,
        "price": p
    },
    index=regions
)
feats = feats[["region", "price", "peak"]]

In [ ]:
for regression, data in regressions.items():
    pred_constr = add_predictor_constr(m, data["pipeline"], feats, d, epsilon=1e-5)

    pred_constr.print_stats()

    data[("Optimization", "#constrs")] = m.NumConstrs + m.NumQConstrs + m.NumGenConstrs
    data[("Optimization", "#vars")] = m.NumVars
    m.Params.NonConvex = 2
    m.Params.OutputFlag = 0
    try:
        start = time()
        m.optimize()
        data[("Optimization", "time")] = time() - start
        if m.Status in (gp.GRB.INFEASIBLE, gp.GRB.INF_OR_UNBD):
            data[("Optimization", "value")] = float('nan')
            data[("Optimization", "viol")] = float('nan')
            data[("Optimization", "error")] = float('nan')
        else:
            data[("Optimization", "value")] = m.ObjVal
            data[("Optimization", "viol")] = m.MaxVio
            data[("Optimization", "error")] = pred_constr.get_error().max()
    except:
        data[("Optimization", "value")] = float('nan')
        data[("Optimization", "viol")] = float('nan')
        data[("Optimization", "error")] = float('nan')
        break
        pass
    pred_constr.remove()

In [ ]:
res = pd.DataFrame.from_dict(regressions, orient='index').drop(["regressor", "pipeline"], axis=1)

In [ ]:
res.columns = pd.MultiIndex.from_tuples(res.columns)


In [ ]:
res.round(3)

Copyright © 2022 Gurobi Optimization, LLC